## Web scraping from wikipedia pages

In [1]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import numpy as np
import json
import re

In [2]:
url1 = "https://en.wikipedia.org/wiki/List_of_Indian_film_actors"
url2 = "https://en.wikipedia.org/wiki/List_of_Indian_film_actresses"
df=[];name=[]

def data_extractor(urls):
    global df 
    
    for url in urls:
        data={}
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html5lib') 

        if soup.find('table',{"class":"infobox biography vcard"}):# Check if the page has infobox
            d=soup.find('table',{"class":"infobox biography vcard"})

            #Extracting required information about actor from infobox
            if(d.find('span',{"class":"bday"})):data['DOB']=d.find('span',{"class":"bday"}).text    

            if(d.find('a',{"class":"image"})):
                li="https://en.wikipedia.org/"+d.find('a',{"class":"image"})['href']
                r1 = requests.get(li);s1 = BeautifulSoup(r1.content, 'html5lib')
                #Extracting links to images from wikimedia pages
                data['Image']='https:'+s1.find('div',{'class':'fullImageLink'}).find('a')['href']

        #Extracting summary from first paragraph in intro section
        data['Summary']=soup.find('p',{'class':''}).text
        df.append(data)
    

def data_scraper(url,len):
    global df;global name
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib') 

    #obtaining list of all actors' page links
    links=soup.find_all('li')[1:len]

    #Obtaining url links for actors page and their name list
    urls=[]
    for link in links:
        p=link.find('a')['href']
        urls.append("https://en.wikipedia.org/"+p)
        name.append(link.find('a').text)
    data_extractor(urls)       

In [3]:
# data_scraper(url2,925)
data_scraper(url1,800)

In [4]:
df=pd.DataFrame(df) 
df["Name"]=name
#Saving the database 
df.to_csv("Movie_celeb_wiki.csv")
df.head()

,DOB,Image,Summary,Name
0,1914-02-01,https://upload.wikimedia.org/wikipedia/commons...,Avtar Kishan Hangal (1 February 1914 – 26 Augu...,A. K. Hangal
1,NaN,https://upload.wikimedia.org/wikipedia/commons...,Aadhi Pinisetty is an Indian film actor who ap...,Aadhi
2,1989-12-23,NaN,Aadi Saikumar (born Aditya Pudipeddi) is an In...,Aadi Pudipeddi
3,1965-03-14,https://upload.wikimedia.org/wikipedia/commons...,Mohammed Aamir Hussain Khan (pronounced [ˈaːmɪ...,Aamir Khan
4,1978-07-21,https://upload.wikimedia.org/wikipedia/commons...,Ashish Chaudhary (born 21 July 1978) is a Bol...,Aashish Chaudhary
